In [3]:
import pandas as pd
import os

os.chdir('C:\\Users\\wzh55\\Desktop\\GitHub\\Guzman-Energy')
data = pd.read_csv("EGPS_PowerCalendar.csv",skiprows=[0])
#I convert all date/time into strings, the split them and compare each part as int
def month_to_number(month):
    if month == "Jan":
        return 1
    if month == "Feb":
        return 2
    if month == "Mar":
        return 3
    if month == "Apr":
        return 4
    if month == "May":
        return 5
    if month == "Jun":
        return 6
    if month == "Jul":
        return 7
    if month == "Aug":
        return 8
    if month == "Sep":
        return 9
    if month == "Oct":
        return 10
    if month == "Nov":
        return 11
    if month == "Dec":
        return 12

def month_to_startandenddate(year, month):
    if month == 2 and year % 4 == 0:
        return 29
    if month == 2 and year % 4 == 1:
        return 28
    if month == 1 or month == 3 or month == 5 or month == 7 or month == 8 or month == 10 or month == 12:
        return 31
    if month == 4 or month == 6 or month == 9 or month == 11:
        return 30
    
def convert_quarter(quarter):
    if quarter == "Q1":
        return 1, 3
    if quarter == "Q2":
        return 4, 6
    if quarter == "Q3":
        return 7, 9
    if quarter == "Q4":
        return 10, 12

def convert_period_to_date(period):
    if "-" in period:
        splited_period = period.split("-")  
        start_year = splited_period[0]
        end_year = splited_period[0]
        start_month = splited_period[1]
        end_month = splited_period[1]        
        start_date = splited_period[2]
        end_date = splited_period[2]
    if len(period) == 7:
        start_year = period[:4]
        end_year = period[:4]
        start_month = month_to_number(period[4:])
        end_month = month_to_number(period[4:])
        start_date = 1                         
        end_date = month_to_startandenddate(end_year, end_month)
    if len(period) == 6:
        start_year = period[:4]
        end_year = period[:4]
        start_month, end_month = convert_quarter(period[4:])
        start_date = 1                         
        end_date = month_to_startandenddate(end_year, end_month)        
    if len(period) == 5:
        start_year = period[:4]
        end_year = period[:4]
        start_month = 1
        end_month = 12
        start_date = 1
        end_date = 31
    starting_date = str(start_month) + "/" + str(start_date) + "/" + str(start_year)
    ending_date = str(end_month) + "/" + str(end_date) + "/" + str(end_year)
    return starting_date, ending_date

def greaterorequaltodate(a, b):
    month_a, date_a, year_a = a.split('/')
    month_b, date_b, year_b = b.split('/')
    if int(year_a) > int(year_b):
        return True
    if int(year_a) < int(year_b):
        return False
    if int(year_a) == int(year_b):
        if int(month_a) > int(month_b):
            return True
        if int(month_a) < int(month_b):
            return False
        if int(month_a) == int(month_b):        
            if int(date_a) > int(date_b):
                return True
            if int(date_a) < int(date_b):
                return False
            if int(date_a) == int(date_b):
                return True

def smallerorequaltodate(a, b):
    month_a, date_a, year_a = a.split('/')
    month_b, date_b, year_b = b.split('/')
    if int(year_a) > int(year_b):
        return False
    if int(year_a) < int(year_b):
        return True
    if int(year_a) == int(year_b):
        if int(month_a) > int(month_b):
            return False
        if int(month_a) < int(month_b):
            return True
        if int(month_a) == int(month_b):        
            if int(date_a) > int(date_b):
                return False
            if int(date_a) < int(date_b):
                return True
            if int(date_a) == int(date_b):
                return True
    
def get_hours(iso, peak_type, period):
    df = pd.read_csv("EGPS_PowerCalendar.csv",skiprows=[0])
    start_date, end_date = convert_period_to_date(period)
    min_index = []
    max_index = []
    for index, row in df.iterrows():
        if greaterorequaltodate(row["StartDate"], start_date) == True:
            min_index.append(index)
        if smallerorequaltodate(row["EndDate"], end_date) == True:
            max_index.append(index)      
    data_after_slicingperiod = df.loc[min(min_index):max(max_index)]
    total_hours = data_after_slicingperiod[peak_type].sum()        
    return [iso,peak_type,start_date,end_date,total_hours]

In [4]:
print(get_hours("ERCOT","Peak","2021Mar"))

[1, 'Peak', '3/1/2021', '3/31/2021', 432]
